# Test Cache in Jupyter Notebook

In [1]:
import helpers.dbg as dbg
import helpers.cache as hcac

# Define computation function

This function will be subjected to cache.

In [2]:
def computation_function(a, b):
    return a + b

## Memory cache

In [3]:
memory_cached_computation = hcac.Cached(computation_function, use_mem_cache=True, use_disk_cache=False)

dbg.dassert_eq(memory_cached_computation(1, 2), 3)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(memory_cached_computation(1, 2), 3)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

print("memory caching checks passed")

Skipping disk cache


memory caching checks passed


## Disk cache

In [4]:
disk_cached_computation = hcac.Cached(computation_function, use_mem_cache=False, use_disk_cache=True)

disk_cached_computation.clear_disk_cache()

dbg.dassert_eq(disk_cached_computation(3, 2), 5)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(disk_cached_computation(3, 2), 5)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

print("disk caching checks passed")

computation_function: clearing disk cache
Skipping memory cache
Skipping memory cache


disk caching checks passed


## Full cache

In [5]:
fully_cached_computation = hcac.Cached(computation_function, use_mem_cache=True, use_disk_cache=True)

fully_cached_computation.clear_disk_cache()

dbg.dassert_eq(fully_cached_computation(3, 2), 5)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(fully_cached_computation(3, 2), 5)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

fully_cached_computation.clear_memory_cache()

dbg.dassert_eq(fully_cached_computation(3, 2), 5)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "disk")

print("full caching checks passed")

computation_function: clearing disk cache
computation_function: clearing memory cache


full caching checks passed
